In [1]:
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO

In [2]:
# The URL from which to download the CSV
RESTAURANT_URL = "https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?fourfour=43nn-pn8j&cacheBust=1711046033&date=20240322&accessType=DOWNLOAD"

# Send a GET request to the URL
response = requests.get(RESTAURANT_URL)
# Check if the request was successful
if response.status_code == 200:
    # Open a file in binary write mode
    with open('restaurant_data.csv', 'wb') as file:
        # Write the content of the response to the file
        file.write(response.content)
    print("CSV file has been downloaded and saved as 'restaurant_data.csv'.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

CSV file has been downloaded and saved as 'restaurant_data.csv'.


In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv('restaurant_data.csv')

df_queens = df[df['BORO'] == 'Queens']

In [4]:
df_queens.shape

(53748, 27)

In [5]:
Queens_df = pd.concat([df_queens], ignore_index=True)

In [6]:
!pip install google-cloud-storage

In [7]:
from google.cloud import storage

In [9]:
# Specify the name of your bucket
bucket_name = 'restaurant-dataset'

#Specify the path to your JSON configuration file
config_file_path = '/content/nyc-restaurants-419717-bdce9165d390.json'

# Authenticate using a service account key
storage_client = storage.Client.from_service_account_json('/content/nyc-restaurants-419717-bdce9165d390.json')

# Access the specified bucket
bucket = storage_client.bucket(bucket_name)

# Specify the folder name within the bucket
folder_name = 'df_raw'

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/QueensInspect.csv')

# Convert DataFrame to CSV
output = StringIO()
Queens_df.to_csv(output, index = False)
data = output.getvalue()
output.close()

try:
    # Upload CSV file to the bucket
    blob.upload_from_string(data, content_type='text/csv')
    print("File uploaded successfully!")
except Exception as e:
    print(f"Error uploading file: {e}")


File uploaded successfully!
